In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
def bash(string,name,queue):
  script = "%s.sh"%name
  with open(script, "w") as text_file:
    text_file.write(string)
  !chmod +x $script
  !qsub -q $queue $script
  #!rm $script

ws = '/beegfs/work/workspace/ws/ho_kezau83-snakemake-0/'

In [ ]:
pro_gen_cog_int = pd.read_table('Full_table_pro_tax_fun_KEGG_COG_for_PCN.tsv', sep='\t', low_memory=False)

In [ ]:
!mkdir -p 2_data_processing/1_PCN/PCN_tables_raw

for x in pro_gen_cog_int.columns[3:119]:
    # get columns for this specific column
    pro_gen_cog_sam = pd.DataFrame(pro_gen_cog_int, columns = ['Genus', 'KEGG_COG', x])
    # print(pro_gen_cog_sam.columns)
    # do sum by groups
    pro_gen_cog_sam_sum = pro_gen_cog_sam.groupby(['Genus', 'KEGG_COG']).sum().reset_index()
    # reshape to wide table
    pro_gen_cog_sam_sum_wide = pro_gen_cog_sam_sum.pivot(index='KEGG_COG', columns='Genus', values= x )
    # write to tables
    pro_gen_cog_sam_sum_wide.to_csv('2_data_processing/1_PCN/PCN_tables_raw/'+ x + '.csv')
    print(x)

In [ ]:
#Step 5. Generate 0 1 PCN tables
### Batch process files - from genus to COG tables, generate an overall PCN and individual sample PCNs ########
!mkdir -p 2_data_processing/1_PCN/PCN_tables_0_1

def findfile(path, tagstr):
    #setup an empty dataframe with same index and column names as all tables
    path = os.path.abspath(path)
    for x in os.listdir(path):
        fulldir = os.path.join(path, x)  # join to absolute path
        if os.path.isfile(fulldir):  # file, match -> print
            # if os.path.split(fulldir)[1]==tagstr   # look for the designated file
            if tagstr in os.path.splitext(fulldir)[1]:  # look for files containing the key word
                print(os.path.join(os.path.abspath(fulldir), fulldir))
                PCN_table_single = pd.read_table(fulldir, delimiter=",").set_index(['KEGG_COG']).fillna(0).T
                if_detected = PCN_table_single.astype('float') > 0
                PCN_table_single['num_detected'] = if_detected.sum(axis=1)
                PCN_table_single_filtered = PCN_table_single.loc[(PCN_table_single.sum(axis=1) != 0), (PCN_table_single.sum(axis=0) != 0)].\
                    sort_values(by ='num_detected', ascending=False).drop(columns='num_detected') #remove empty columns for individual files
                PCN_table_single_filtered = PCN_table_single_filtered.astype(
                    'float') > 0  # replace values with presence or not (1 or 0), it will give a True or False value
                PCN_table_single_filtered = PCN_table_single_filtered.astype('float')  # replace True or False with 1 or 0
                PCN_table_single_filtered_sum = pd.DataFrame(PCN_table_single_filtered.sum(axis=0)).rename(columns={0: 'sum_num'}).T
                PCN_table_single_filtered_sum_col = pd.concat([PCN_table_single_filtered, PCN_table_single_filtered_sum]).sort_values(by='sum_num', axis=1,
                                                                                            ascending=False).drop(
                    index='sum_num') # get sorted PCN for each individual sample
                PCN_table_single_filtered_sum_col.to_csv('2_data_processing/1_PCN/PCN_tables_0_1/' + os.path.splitext(x)[0] + '.csv')

findfile('2_data_processing/1_PCN/PCN_tables_raw/', '.csv')

In [ ]:
## Part 2. Degree distributions
# Degree distributions
!mkdir -p 2_data_processing/2_Degree_distribution/Taxon
!mkdir -p 2_data_processing/2_Degree_distribution/Function

def findfile(path, tagstr):
    path = os.path.abspath(path)

    for x in os.listdir(path):

        fulldir = os.path.join(path, x)  # join to absolute path

        if os.path.isfile(fulldir):  # file, match -> print
            # if os.path.split(fulldir)[1]==tagstr   # look for the designated file
            if tagstr in os.path.splitext(fulldir)[1]:  # look for files containing the key word
                print(os.path.join(os.path.abspath(fulldir), fulldir))

                ######## Calculation starts here ###############################
                origin_data = pd.read_table(fulldir, delimiter=",", index_col=0)           #read a PCN_0_1 table

                taxon_degree_distribution = pd.DataFrame()
                COG_degree_distribution = pd.DataFrame()

                # calculate number of nodes k degree
                taxon_degree_distribution['k'] = origin_data.sum(axis=1)
                COG_degree_distribution['k'] = origin_data.sum(axis=0)

                # calculate pk fraction of nodes
                taxon_pk = pd.DataFrame(columns=['k'])
                taxon_pk['pk'] = taxon_degree_distribution['k'].value_counts() / len(taxon_degree_distribution.index)
                taxon_pk['k'] = taxon_pk.index
                taxon_degree_distribution = taxon_pk.merge(taxon_degree_distribution, on='k')

                COG_pk = pd.DataFrame(columns=['k'])
                COG_pk['pk'] = COG_degree_distribution['k'].value_counts() / len(COG_degree_distribution.index)
                COG_pk['k'] = COG_pk.index
                COG_degree_distribution = COG_pk.merge(COG_degree_distribution, on='k')

                taxon_degree_distribution.to_csv('2_data_processing/2_Degree_distribution/Taxon/'+ x)
                COG_degree_distribution.to_csv('2_data_processing/2_Degree_distribution/Function/'+ x)


findfile('2_data_processing/1_PCN/PCN_tables_0_1', '.csv')

In [ ]:
file_list = os.listdir('2_data_processing/1_PCN/PCN_tables_raw/')

In [ ]:
file_list = ['Seifert_Hammel_Nr001.csv']

In [ ]:
# Set the index properly to SET NAME
### Part 3. deep MP dataset, all genera, functional distance ###
# the results will be used for FR calculation, but not for dij visualization
string = '''
#PBS -l nodes=1:ppn=14
#PBS -l walltime=8:00:00
#PBS -l mem=64gb
#PBS -S /bin/bash
source /beegfs/work/workspace/ws/ho_kezau83-conda-0/conda/etc/profile.d/conda.sh
conda activate fr-metalab

python /beegfs/work/workspace/ws/ho_kezau83-snakemake-0/untitled.py %s

conda deactivate
'''
!mkdir -p 2_data_processing/3_PCN_Functional_distance/all/norm/matrix/
!mkdir -p 2_data_processing/3_PCN_Functional_distance/all/norm/list

for file in file_list:
    file = ws + '2_data_processing/1_PCN/PCN_tables_raw/%s' % file
    bash(string%(file),'%s_fr-metalab'%file,'short')

In [ ]:
!qstat -u ho_kezau83

In [12]:
#The index means to get the sample name, this can be improve


# Calculates functional redundancy following this equation:
# FR = ∑(N,i=1)∑(N,j=1)(1-dij)pipj

## loop for all samples - deep metaproteomics
######## Batch process files - calculating Dij for each sample ########
!mkdir -p 2_data_processing/4_Functional_redundancy/

genus_table_original = pd.read_table("Taxonomy_table_genus_level_metaproteomics.tsv",
                                     delimiter="\t", index_col=0)
genus_table_p_matrix = genus_table_original.div(genus_table_original.sum(axis=0), axis=1)

## loop for all samples - deep metaproteomics
######## Batch process files - calculating Dij for each sample ########
def findfile(path, tagstr):
    Record_FR = pd.DataFrame()
    Record_FR_norm = pd.DataFrame()
    count = 0

    path = os.path.abspath(path)

    for x in os.listdir(path):

        fulldir = os.path.join(path, x)  # join to absolute path

        if os.path.isfile(fulldir):  # file, match -> print
            # if os.path.split(fulldir)[1]==tagstr   # look for the designated file
            if tagstr in os.path.splitext(fulldir)[1]:  # look for files containing the key word
                print(os.path.join(os.path.abspath(fulldir), fulldir))

                ######## Calculation starts here ###############################
                dij_table_data = pd.read_table(fulldir, delimiter=",", index_col=0)
                dij_table_data_indexed = pd.concat(
                    [dij_table_data, dij_table_data["VS"].str.split("_vs_", n=1, expand=True)], axis=1).rename(
                    columns={0: "i", 1: "j"})
                # index to the sample column in the genus_table_p_matrix
                genus_table_p_sample_i = genus_table_p_matrix[[x[-24:-4]]].rename(columns={x[-24:-4]: 'Abundance'})
                genus_table_p_sample_j = genus_table_p_matrix[[x[-24:-4]]].rename(columns={x[-24:-4]: 'Abundance'})

                # assign pi and pj values to the data table
                genus_table_p_sample_i = genus_table_p_sample_i.reset_index().rename(columns={'Name': "i"})
                genus_table_p_sample_j = genus_table_p_sample_j.reset_index().rename(columns={'Name': "j"})
                dij_table_trial_indexed_p = dij_table_data_indexed.merge(genus_table_p_sample_i, on='i').rename(
                    columns={'Abundance': 'pi'})
                dij_table_trial_indexed_p = dij_table_trial_indexed_p.merge(genus_table_p_sample_j, on='j').rename(
                    columns={'Abundance': 'pj'})

                # Calculate (1-dij)pipj
                FR = 0
                GSI = 0
                for x in dij_table_trial_indexed_p.index:
                    FR_ij = (1 - dij_table_trial_indexed_p.loc[x, 'dij_list']) * dij_table_trial_indexed_p.loc[
                        x, 'pi'] * dij_table_trial_indexed_p.loc[x, 'pj']
                    FR = FR + FR_ij * 2
                    # Taxonomic diversity: Gini-Simpson index
                    GSI_ij = dij_table_trial_indexed_p.loc[x, 'pi'] * dij_table_trial_indexed_p.loc[x, 'pj']
                    GSI = GSI + GSI_ij * 2
                print(FR)
                print(GSI)
                Record_FR.loc[count, 0] = fulldir[-9:-4]
                Record_FR.loc[count, 1] = FR
                Record_FR.loc[count, 2] = FR/GSI # Functional redundancy normalized by taxonomic diveristy
                Record_FR.loc[count, 3] = GSI
                Record_FR.loc[count, 4] = GSI-FR
                count = count + 1

    Record_FR.rename(columns={0: "Sample", 1: "FR", 2: "nFR", 3: "TD", 4: "FD"}).to_csv('2_data_processing/4_Functional_redundancy/' + 'Record_FR_PCN.csv')


findfile('2_data_processing/3_PCN_Functional_distance/all/norm/list', '.csv')

/beegfs/work/workspace/ws/ho_kezau83-snakemake-0/2_data_processing/3_PCN_Functional_distance/all/norm/list/all_norm_dij_list_Seifert_Hammel_Nr084.csv
0.07486511593385148
0.9735462027583618
/beegfs/work/workspace/ws/ho_kezau83-snakemake-0/2_data_processing/3_PCN_Functional_distance/all/norm/list/all_norm_dij_list_Seifert_Hammel_Nr025.csv
0.06193065053855085
0.9576402783304436
/beegfs/work/workspace/ws/ho_kezau83-snakemake-0/2_data_processing/3_PCN_Functional_distance/all/norm/list/all_norm_dij_list_Seifert_Hammel_Nr077.csv
0.08717395947468791
0.9712188849236459
/beegfs/work/workspace/ws/ho_kezau83-snakemake-0/2_data_processing/3_PCN_Functional_distance/all/norm/list/all_norm_dij_list_Seifert_Hammel_Nr102.csv
0.07594424407630994
0.9804484890978615
/beegfs/work/workspace/ws/ho_kezau83-snakemake-0/2_data_processing/3_PCN_Functional_distance/all/norm/list/all_norm_dij_list_Seifert_Hammel_Nr043.csv
0.06859066217184812
0.982237217301352
/beegfs/work/workspace/ws/ho_kezau83-snakemake-0/2_data_

In [ ]:
x[-16:-4]

In [7]:
x = "/beegfs/work/workspace/ws/ho_kezau83-snakemake-0/2_data_processing/3_PCN_Functional_distance/all/norm/list/all_norm_dij_list_Seifert_Hammel_Nr019.csv"

genus_table_original = pd.read_table("Taxonomy_table_genus_level_metaproteomics.tsv",
                                     delimiter="\t", index_col=0)
genus_table_p_matrix = genus_table_original.div(genus_table_original.sum(axis=0), axis=1)
genus_table_p_sample_i = genus_table_p_matrix[[x[-24:-4]]].rename(columns={x[-29:-4]: 'Abundance'})



In [8]:
genus_table_p_matrix

,Seifert_Hammel_Nr001,Seifert_Hammel_Nr010,Seifert_Hammel_Nr100,Seifert_Hammel_Nr101,Seifert_Hammel_Nr102,Seifert_Hammel_Nr103,Seifert_Hammel_Nr104,Seifert_Hammel_Nr105,Seifert_Hammel_Nr011,Seifert_Hammel_Nr113,...,Seifert_Hammel_Nr090,Seifert_Hammel_Nr091,Seifert_Hammel_Nr092,Seifert_Hammel_Nr093,Seifert_Hammel_Nr094,Seifert_Hammel_Nr095,Seifert_Hammel_Nr096,Seifert_Hammel_Nr097,Seifert_Hammel_Nr098,Seifert_Hammel_Nr099
Name,,,,,,,,,,,,,,,,,,,,,
12844,0.001885,0.001446,0.000843,0.001112,0.001268,0.001243,0.000552,0.000876,0.001484,0.001487,...,0.000831,0.000986,0.000945,0.001519,0.000468,0.001317,9.032201e-04,0.001833,0.001359,0.000921
51-20,0.000000,0.000000,0.000000,0.000000,0.000002,0.000004,0.000000,0.000000,0.000011,0.000000,...,0.000003,0.000000,0.000004,0.000002,0.000002,0.000000,9.023978e-07,0.000000,0.000001,0.000000
AC2028,0.005556,0.004067,0.003233,0.004446,0.003719,0.003703,0.004875,0.003752,0.006204,0.004769,...,0.002944,0.005266,0.003947,0.004840,0.001920,0.006327,3.117098e-03,0.002873,0.003173,0.003386
Absicoccus,0.000029,0.000014,0.000039,0.000040,0.000050,0.000017,0.000035,0.000021,0.000112,0.000166,...,0.000015,0.000048,0.000051,0.000047,0.000032,0.000290,3.396622e-05,0.000014,0.000042,0.000033
Acetatifactor,0.002927,0.002532,0.001831,0.002515,0.003319,0.002996,0.002827,0.003620,0.004093,0.002437,...,0.002133,0.002851,0.003305,0.004343,0.001405,0.001916,1.937915e-03,0.002741,0.002448,0.002112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WCHB1-69,0.000092,0.000139,0.000099,0.000124,0.000104,0.000233,0.000132,0.000175,0.000104,0.000034,...,0.000183,0.000060,0.000055,0.000094,0.000041,0.000032,3.063599e-05,0.000059,0.000019,0.000042
Weimerbacter,0.000654,0.000330,0.000330,0.000568,0.000374,0.000365,0.000830,0.000274,0.001204,0.000622,...,0.000666,0.000479,0.000446,0.000481,0.000270,0.001556,4.501783e-04,0.000325,0.001013,0.000538
XBB2008,0.000014,0.000000,0.000019,0.000000,0.000022,0.000000,0.000000,0.000002,0.000002,0.000000,...,0.000000,0.000018,0.000000,0.000000,0.000000,0.000018,1.524981e-05,0.000000,0.000003,0.000000


In [ ]:
!ls 2_data_processing/3_PCN_Functional_distance/all/norm/matrix/ | 

In [ ]:
!rm -r 2_data_processing/

In [11]:
file_path = "/beegfs/work/workspace/ws/ho_kezau83-snakemake-0/2_data_processing/3_PCN_Functional_distance/all/norm/list/all_norm_dij_list_Seifert_Hammel_Nr019.csv"
x = file_path[-24:-4]
x

'Seifert_Hammel_Nr019'

In [ ]:
!rename ifert Seifert 2_data_processing/3_PCN_Functional_distance/all/norm/list/*.csv